In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [5]:
data = pd.read_csv("input/data2.csv")

In [6]:
data

,state,district,democA,voteA,expendA,expendB,prtystrA,lexpendA,lexpendB,shareA
0,AL,7,1,68,328.30,8.74,41,5.793916,2.167567,97.41
1,AK,1,0,62,626.38,402.48,60,6.439952,5.997638,60.88
2,AZ,2,1,73,99.61,3.07,55,4.601233,1.120048,97.01
3,AZ,3,0,69,319.69,26.28,64,5.767352,3.268846,92.40
4,AR,3,0,75,159.22,60.05,66,5.070293,4.095244,72.61
...,...,...,...,...,...,...,...,...,...,...
168,WV,4,0,39,32.04,152.27,42,3.466954,5.025662,17.38
169,WI,3,1,32,22.63,359.80,53,3.119100,5.885551,5.92
170,WI,5,0,36,197.46,1278.53,36,5.285536,7.153463,13.38
171,WI,7,0,38,202.59,450.72,46,5.311189,6.110837,31.01


In [38]:
col = data.columns.values
col

array(['age', 'soph', 'junior ', 'senior', 'senior5', 'male', 'campus',
       'business', 'engineer', 'colGPA', 'hsGPA', 'ACT', 'job19', 'job20',
       'drive', 'bike', 'walk', 'voluntr', 'PC', 'greek', 'car',
       'siblings', 'bgfriend', 'clubs', 'skipped', 'alcohol', 'gradMI',
       'fathcoll', 'mothcoll'], dtype=object)

### Question.C
Estimate the multiple regression equation

$$
vote(A)=β_0+β_1 log(expend(A))+β_2 log(expend(B))+β_3 prystr(A)+μ
$$

Do A’s expenditures affect the outcome? What about B’s expenditures? Can you use these results to test the hypothesis in part (b)?



In [7]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [47]:
from sklearn.linear_model import LinearRegression

data_c = data.loc[:,['voteA','expendA',"expendB", "prtystrA"]]
data_c.columns = ['voteA','log_exA',"log_exB", "prtystrA"]

data_c["log_exA"] = np.log(data["expendA"])
data_c["log_exB"] = np.log(data["expendB"])



x_c = data_c.loc[:,'log_exA':"prtystrA"]
y = data_c[['voteA']]

LR_c = LinearRegression()
LR_c.fit(x_c,y)

coef_c = LR_c.coef_[0]

print('回帰係数 β1: %.3f, β2: %.3f, β3: %.3f, ' %(coef_c[0],coef_c[1],coef_c[2]))
print()
print('切片 β0: %.3f' %LR_c.intercept_)
print()
print('y=  %.3f + %.3flog(expend(A)) + %.3flog(expend(B))+ %.3fprtystr(A) ' % (LR_c.intercept_,coef_c[0],coef_c[1],coef_c[2]))
print()
print('決定係数 R^2： ', LR_c.score(x_c, y))

回帰係数 β1: 6.081, β2: -6.616, β3: 0.152, 

切片 β0: 45.088

y=  45.088 + 6.081log(expend(A)) + -6.616log(expend(B))+ 0.152prtystr(A) 

決定係数 R^2：  0.7925010212651955


In [48]:
x_add_const = sm.add_constant(x_c)
model_sm = sm.OLS(y, x_add_const).fit()
print(model_sm.summary())

                            OLS Regression Results                            
Dep. Variable:                  voteA   R-squared:                       0.793
Model:                            OLS   Adj. R-squared:                  0.789
Method:                 Least Squares   F-statistic:                     215.2
Date:                Thu, 20 May 2021   Prob (F-statistic):           1.80e-57
Time:                        18:01:46   Log-Likelihood:                -596.88
No. Observations:                 173   AIC:                             1202.
Df Residuals:                     169   BIC:                             1214.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         45.0879      3.927     11.482      0.0

### Question.D



I think
$$
H_0  ：β_1+β_2=0　,　 H_1  ：β_1+β_2≠0
$$
so,
$$
θ = β_1+β_2 ,  β_1=θ-β_2
$$

Estimate the multiple regression equation

$$
vote(A)=β_0+(θ-β_2) log(expend(A))+β_2 log(expend(B))+β_3 prystr(A)+μ
$$$$
        =β_0+θ log(expendA)+β_2 (log(expendB)-log(expendA))+β_3 prystr(A)+μ
$$




In [52]:
data_d = data_c.copy()
data_d.columns = ['voteA','log_exA',"dif(log_exB,A)", "prtystrA"]
data_d["dif(log_exB,A)"] = data_c["log_exB"] - data_c["log_exA"]

x_d = data_d.loc[:,'log_exA':"prtystrA"]
#y = data_c[['voteA']]

LR_d = LinearRegression()
LR_d.fit(x_d,y)

"""
plt.plot(x, y, 'o')
plt.plot(x, LR_c.predict(x), linestyle="solid")
plt.show()
"""
coef_d = LR_d.coef_[0]

print('回帰係数 θ: %.3f, β2: %.3f, β3: %.3f, ' %(coef_d[0],coef_d[1],coef_d[2]))
print()
print('切片 β0: %.3f' %LR_d.intercept_)
print()
print('y=  %.3f + %.3flog(expend(A)) + %.3fdif(log_exB,A)+ %.3fprtystr(A) ' % (LR_d.intercept_,coef_d[0],coef_d[1],coef_d[2]))
print()
print('決定係数 R^2： ', LR_d.score(x_d, y))


回帰係数 θ: -0.534, β2: -6.616, β3: 0.152, 

切片 β0: 45.088

y=  45.088 + -0.534log(expend(A)) + -6.616dif(log_exB,A)+ 0.152prtystr(A) 

決定係数 R^2：  0.7925010212651955


In [54]:
x_add_const = sm.add_constant(x_d)
model_sm = sm.OLS(y, x_add_const).fit()
print(model_sm.summary())

                            OLS Regression Results                            
Dep. Variable:                  voteA   R-squared:                       0.793
Model:                            OLS   Adj. R-squared:                  0.789
Method:                 Least Squares   F-statistic:                     215.2
Date:                Thu, 20 May 2021   Prob (F-statistic):           1.80e-57
Time:                        20:24:54   Log-Likelihood:                -596.88
No. Observations:                 173   AIC:                             1202.
Df Residuals:                     169   BIC:                             1214.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             45.0879      3.927     11.